In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

input_dir = '../datasets/crop/mine'
output_dir = '../datasets/augmented/mine'
os.makedirs(output_dir, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

n_aug_per_image = 10
image_size = (128, 128)

for fname in os.listdir(input_dir):
    if fname.lower().endswith(('.jpg', '.png')):
        path = os.path.join(input_dir, fname)
        img = cv2.imread(path)
        img = cv2.resize(img, image_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = np.expand_dims(img, 0)

        prefix = os.path.splitext(fname)[0]
        i = 0
        for batch in datagen.flow(x, batch_size=1):
            aug_img = batch[0].astype(np.uint8)
            aug_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
            out_path = os.path.join(output_dir, f"{prefix}_aug{i}.jpg")
            cv2.imwrite(out_path, aug_bgr)
            i += 1
            if i >= n_aug_per_image:
                break

print(f"✅ 증강 완료: {len(os.listdir(output_dir))}개의 이미지가 생성되었습니다.")


✅ 증강 완료: 190개의 이미지가 생성되었습니다.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

input_dir = '../datasets/crop/background'
aug_dir = '../datasets/augmented/background'
os.makedirs(aug_dir, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

n_aug = 5
image_size = (128, 128)

for fname in os.listdir(input_dir):
    if fname.lower().endswith(('.jpg', '.png')):
        path = os.path.join(input_dir, fname)
        img = cv2.imread(path)
        img = cv2.resize(img, image_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = np.expand_dims(img, 0)

        prefix = os.path.splitext(fname)[0]
        i = 0
        for batch in datagen.flow(x, batch_size=1):
            aug_img = batch[0].astype(np.uint8)
            aug_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
            out_path = os.path.join(aug_dir, f"{prefix}_aug{i}.jpg")
            cv2.imwrite(out_path, aug_bgr)
            i += 1
            if i >= n_aug:
                break

print("✅ 배경 이미지 증강 완료.")

✅ 배경 이미지 증강 완료.


In [7]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def extract_hog_lbp(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    
    hog_feat = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    
    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)
    
    return np.concatenate([hog_feat, lbp_hist])

mine_dirs = [
    '../datasets/crop/mine',
    '../datasets/augmented/mine'
]
bg_dirs = [
    '../datasets/crop/background',
    '../datasets/augmented/background'
]

X, y = [], []

for d in mine_dirs:
    for fname in os.listdir(d):
        if fname.lower().endswith(('.jpg', '.png')):
            X.append(extract_hog_lbp(os.path.join(d, fname)))
            y.append(1)

for d in bg_dirs:
    for fname in os.listdir(d):
        if fname.lower().endswith(('.jpg', '.png')):
            X.append(extract_hog_lbp(os.path.join(d, fname)))
            y.append(0)

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_leaf=3,
    class_weight='balanced',
    random_state=42
)
clf.fit(X_train, y_train)

probs = clf.predict_proba(X_test)[:, 1]
threshold = 0.4
y_pred = (probs > threshold).astype(int)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.79      0.86        68
           1       0.75      0.93      0.83        46

    accuracy                           0.85       114
   macro avg       0.85      0.86      0.85       114
weighted avg       0.87      0.85      0.85       114



In [8]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

def extract_hog_lbp(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    
    hog_feat = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    
    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)
    
    return np.concatenate([hog_feat, lbp_hist])

mine_dirs = [
    '../datasets/crop/mine',
    '../datasets/augmented/mine'
]
bg_dirs = [
    '../datasets/crop/background',
    '../datasets/augmented/background'
]

X, y = [], []

for d in mine_dirs:
    for fname in os.listdir(d):
        if fname.lower().endswith(('.jpg', '.png')):
            X.append(extract_hog_lbp(os.path.join(d, fname)))
            y.append(1)

for d in bg_dirs:
    for fname in os.listdir(d):
        if fname.lower().endswith(('.jpg', '.png')):
            X.append(extract_hog_lbp(os.path.join(d, fname)))
            y.append(0)

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=float(np.sum(y_train == 0)) / np.sum(y_train == 1),
    eval_metric='logloss',
    random_state=42
)
clf.fit(X_train, y_train)

probs = clf.predict_proba(X_test)[:, 1]
threshold = 0.4
y_pred = (probs > threshold).astype(int)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85        68
           1       0.76      0.83      0.79        46

    accuracy                           0.82       114
   macro avg       0.82      0.82      0.82       114
weighted avg       0.83      0.82      0.83       114

